# Modeling

### Pre-Processing

Import required libraries.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", None)
plt.style.use('ggplot')

Read in cleaned dataframe.

In [2]:
housedf = pd.read_csv('Ames_HousePrice_processed.csv', index_col=0)

Check to see if dataframe has been read as intended.

In [3]:
housedf.head()

,PID,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,ExteriorSF,AgeAtSale,TotalFullBath,TotalHalfBath,GarageAgeAtSale,Prop_Addr,MA_Ownr1,MA_Ownr2,Latitude,Longitude
0,909176150,856,126000,30,RL,0.0,7890,Pave,No alley,Reg,Lvl,AllPub,Corner,Gtl,SWISU,Norm,Norm,1Fam,1Story,6,6,1939,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,238.0,Unf,0.0,618.0,856.0,GasA,TA,Y,SBrkr,856,0,0,1.0,0.0,1,0,2,1,TA,4,Typ,1,Gd,Detchd,Unf,2.0,399.0,TA,TA,Y,0,0,0,0,166,0,No pool,No fence,No misc feature,0,3,2010,WD,Normal,7034,71,2.0,0.0,71.0,436 HAYWARD AVE,"GREINER, RODNEY & CELIA",NaN,42.018564,-93.651619
1,905476230,1049,139500,120,RL,42.0,4235,Pave,No alley,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,TwnhsE,1Story,5,5,1984,1984,Gable,CompShg,HdBoard,HdBoard,BrkFace,149.0,Gd,TA,CBlock,Gd,TA,Mn,GLQ,552.0,ALQ,393.0,104.0,1049.0,GasA,TA,Y,SBrkr,1049,0,0,1.0,0.0,2,0,2,1,Gd,5,Typ,0,No fireplace,Attchd,Fin,1.0,266.0,TA,TA,Y,0,105,0,0,0,0,No pool,No fence,No misc feature,0,2,2009,WD,Normal,3186,25,3.0,0.0,25.0,3416 WEST ST,"HUBERT, JOHN F & GRETTA R",NaN,42.024855,-93.663671
2,911128020,1001,124900,30,C (all),60.0,6060,Pave,No alley,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1Story,5,9,1930,2007,Hip,CompShg,MetalSd,MetalSd,None,0.0,Gd,TA,BrkTil,TA,TA,No,ALQ,737.0,Unf,0.0,100.0,837.0,GasA,Ex,Y,SBrkr,1001,0,0,0.0,0.0,1,0,2,1,Gd,5,Typ,0,No fireplace,Detchd,Unf,1.0,216.0,TA,Po,N,154,0,42,86,0,0,No pool,No fence,No misc feature,0,11,2007,WD,Normal,5059,77,1.0,0.0,77.0,320 S 2ND ST,"SKLENAR, TIM","CARNEY, SARAH",42.021548,-93.614068
3,535377150,1039,114000,70,RL,80.0,8146,Pave,No alley,Reg,Lvl,AllPub,Corner,Gtl,OldTown,Norm,Norm,1Fam,2Story,4,8,1900,2003,Gable,CompShg,MetalSd,MetalSd,None,0.0,Gd,Gd,BrkTil,Fa,TA,No,Unf,0.0,Unf,0.0,405.0,405.0,GasA,Gd,Y,SBrkr,717,322,0,0.0,0.0,1,0,2,1,TA,6,Typ,0,No fireplace,Detchd,Unf,1.0,281.0,TA,TA,N,0,0,168,0,111,0,No pool,No fence,No misc feature,0,5,2009,WD,Normal,7429,109,1.0,0.0,69.0,1524 DOUGLAS AVE,"MILLARD, TAMMY",NaN,42.037391,-93.612207
4,534177230,1665,227000,60,RL,70.0,8400,Pave,No alley,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,2Story,8,6,2001,2001,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,TA,No,GLQ,643.0,Unf,0.0,167.0,810.0,GasA,Ex,Y,SBrkr,810,855,0,1.0,0.0,2,1,3,1,Gd,6,Typ,0,No fireplace,Attchd,Fin,2.0,528.0,TA,TA,Y,0,45,0,0,0,0,No pool,No fence,No misc feature,0,11,2009,WD,Normal,7590,8,3.0,1.0,8.0,2304 FILLMORE AVE,"SEBBAG, LIONEL","LEVIN, ALISON P",42.044554,-93.631818


In [4]:
housedf.shape

(2579, 90)

In [9]:
housedf.isnull().values.any()

True

Make a copy of the dataframe for additional pre-processing for modeling purposes.

In [5]:
housedfcopy = housedf.copy()

In [6]:
housedfcopy.shape

(2579, 90)

Drop `PID`, `MA_Ownr1`, `MA_Ownr2`, `Prop_Addr` columns from dataframe:

- `PID` is an identification number that was used to join with `Real Estate Data` in the data preprocessing notebook. Since it adds no value in the modeling step, we need to drop.
- `MA_Ownr1` and `MA_Ownr2` are not expected to add value to the modeling step, we need to drop. 
- `Prop_Addr` has multiple missing values, and we can capture the same information with `Latitude`, `Longitude` and `Neighborhood` features, we need to drop.

In [7]:
housedfcopy = housedfcopy.drop(labels = ["PID", "MA_Ownr1", "MA_Ownr2", 'Prop_Addr'], axis = 1)

`Latitude` and `Longitude` have several missing values. Impute with the most frequent values in these feature columns.

In [8]:
housedfcopy["Latitude"] = housedfcopy["Latitude"].fillna(housedfcopy["Latitude"].mode()[0])
housedfcopy["Longitude"] = housedfcopy["Longitude"].fillna(housedfcopy["Longitude"].mode()[0])

Check to see if all null values have been removed prior to modeling.

In [10]:
housedfcopy.isnull().values.any()

False

# Feature Exploration

Use the following methods for feature exploration and discovery of which are more important:

- Discover features with the least amount of variance with `VarianceThreshold()`. If the feature has low variance, this feature may not help us identify an increase in `SalePrice`.
- Univariate feature selection with F test with `f_regression`. If the feature has a high F value and low p-value, that means that it will be significant in modeling.

### `VarianceThreshold()`

In [30]:
import sklearn.feature_selection as fs
varied_features = fs.VarianceThreshold(threshold = 0.25)
x_varied = varied_features.fit_transform(X)
x_varied.shape

(2579, 71)

In [34]:
fs.VarianceThreshold.get_support(varied_features)

array([ True,  True,  True,  True,  True, False, False,  True,  True,
       False,  True, False,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False,  True, False,  True,  True,  True,
        True,  True, False,  True, False,  True, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False, False])

In [37]:
concol = [column for column in X.columns 
          if column not in X.columns[varied_features.get_support()]]

for features in concol:
    print(features)

Street
Alley
Utilities
LandSlope
Condition2
Heating
CentralAir
BsmtHalfBath
HalfBath
KitchenAbvGr
PoolQC
MiscFeature
Latitude
Longitude


#### Observations from feature `VarianceThreshold` analysis:

The following features have low variance and may not have a distinguishable effect on `SalePrice`:

- Features Regarding Location:
    - `Street`, `Latitude`, `Longitude`, `Condition2`
- Features Regarding Lot:
    - `Alley`, `LandSlope`
- Features Regarding Utilities:
    - `Utilities`, `Heating`, `CentralAir`
- Features Regarding Bathrooms:
    - `BsmtHalfBath`, `Halfbath`
- Features Regarding Misc:
    - `KitchAbvGr`, `MiscFeature`
    
This indicates we may want to drop these features or combine them with another feature. 

### `f_regression`

In [159]:
skb = fs.SelectKBest(score_func = fs.f_regression, k = 'all')

In [160]:
best_feats = skb.fit_transform(X, y)

In [164]:
for i in range(len(skb.pvalues_)):
 print('%s: %f' % (X.columns[i], skb.pvalues_[i]))

GrLivArea: 0.000000
MSSubClass: 0.000017
MSZoning: 0.000000
LotFrontage: 0.000000
LotArea: 0.000000
Street: 0.010395
Alley: 0.000000
LotShape: 0.000000
LandContour: 0.051559
Utilities: 0.083717
LotConfig: 0.001352
LandSlope: 0.000062
Neighborhood: 0.000000
Condition1: 0.000002
Condition2: 0.039810
BldgType: 0.003563
HouseStyle: 0.000000
OverallQual: 0.000000
OverallCond: 0.000000
YearBuilt: 0.000000
YearRemodAdd: 0.000000
RoofStyle: 0.000000
RoofMatl: 0.000000
Exterior1st: 0.000000
Exterior2nd: 0.000000
MasVnrType: 0.000002
MasVnrArea: 0.000000
ExterQual: 0.000000
ExterCond: 0.000000
Foundation: 0.000000
BsmtQual: 0.000000
BsmtCond: 0.000000
BsmtExposure: 0.000000
BsmtFinType1: 0.000000
BsmtFinSF1: 0.000000
BsmtFinType2: 0.010912
BsmtFinSF2: 0.249295
BsmtUnfSF: 0.000000
TotalBsmtSF: 0.000000
Heating: 0.000102
HeatingQC: 0.000000
CentralAir: 0.000000
Electrical: 0.000000
1stFlrSF: 0.000000
2ndFlrSF: 0.000000
LowQualFinSF: 0.117548
BsmtFullBath: 0.000000
BsmtHalfBath: 0.086452
FullBath: 

### Observations from `f_regression` p-values.

The following features have a p-value of 0.05 or higher, so these will not be significant on prediciting `SalePrice`:

- `LandContour`: 0.051559
- `Utilities`: 0.083717
- `BsmtFinSF2`: 0.249295
- `LowQualFinSF`: 0.117548
- `BsmtHalfBath`: 0.086452
- `3SsnPorch`: 0.112386
- `PoolArea`: 0.117289
- `MiscVal`: 0.338131
- `MoSold`: 0.518331
- `YrSold`: 0.948975
- `SaleType`: 0.962618

In [196]:
skb = fs.SelectKBest(score_func = fs.f_regression, k = 20)

In [197]:
best_feats = skb.fit_transform(X, y)

In [198]:
concol = [column for column in X.columns 
          if column in X.columns[skb.get_support()]]

for features in concol:
    print(features)

GrLivArea
OverallQual
YearBuilt
YearRemodAdd
MasVnrArea
ExterQual
BsmtQual
BsmtFinSF1
TotalBsmtSF
1stFlrSF
FullBath
KitchenQual
TotRmsAbvGrd
Fireplaces
GarageFinish
GarageCars
GarageArea
AgeAtSale
TotalFullBath
GarageAgeAtSale


If focusing on non-square footage features, features that have the best F-score are the following **in order of importance**:

- `OverallQual`
- `GarageCars`
- `ExterQual`
- `BsmtQual`
- `KitchenQual`
- `TotalFullBath`
- `GarageFinish`
- `FullBath`
- `TotRmsAbvGrd`
- `FirePlaces`
- `TotalFullBath`

# Feature Engineering

In [199]:
housedfcopy.columns

Index(['GrLivArea', 'SalePrice', 'MSSubClass', 'MSZoning', 'LotFrontage',
       'LotArea', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd',
       'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageFinish',
       'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive',
       'Wood

In [200]:
housedfcopy.shape

(2579, 86)

# Modeling

Find the categorical variables for label enconding. Anything as `object` type, use for label enconding.

In [12]:
dtypecols = housedfcopy.columns.to_series().groupby(housedfcopy.dtypes).groups
dtypecols

{int64: ['GrLivArea', 'SalePrice', 'MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'ExteriorSF', 'AgeAtSale'], float64: ['LotFrontage', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'GarageCars', 'GarageArea', 'TotalFullBath', 'TotalHalfBath', 'GarageAgeAtSale', 'Latitude', 'Longitude'], object: ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'Heating

In [13]:
from sklearn.preprocessing import LabelEncoder

housedfcopy[['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 
            'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 
            'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 
            'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 
            'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 
            'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 
            'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 
             'SaleCondition']] = housedfcopy[['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 
                                              'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 
                                              'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 
                                              'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 
                                              'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 
                                              'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 
                                              'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 
                                              'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 
                                              'Fence', 'MiscFeature', 'SaleType', 
                                              'SaleCondition']].apply(LabelEncoder().fit_transform)

In [29]:
housedfcopy[['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 
            'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 
            'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 
            'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 
            'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 
            'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 
            'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 
             'SaleCondition']].head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,5,1,1,3,3,0,0,0,21,2,2,0,2,1,0,13,14,2,3,4,1,5,5,3,5,6,1,4,1,3,4,6,2,5,2,5,5,2,3,4,1,9,4
1,5,1,1,3,3,0,4,0,7,2,2,4,2,1,0,6,6,1,2,4,1,2,5,2,2,0,1,4,1,3,2,6,3,1,0,5,5,2,3,4,1,9,4
2,1,1,1,3,3,0,4,0,11,2,2,0,2,3,0,8,8,2,2,4,0,5,5,3,0,6,1,0,1,3,2,6,3,5,2,5,4,0,3,4,1,9,4
3,5,1,1,3,3,0,0,0,20,2,2,0,5,1,0,8,8,2,2,2,0,1,5,3,6,6,1,2,1,3,4,6,3,5,2,5,5,0,3,4,1,9,4
4,5,1,1,3,3,0,4,0,17,2,2,0,5,1,0,12,13,2,2,4,2,2,5,3,2,6,1,0,1,3,2,6,3,1,0,5,5,2,3,4,1,9,4


In [15]:
housedfcopy.isnull().values.any()

False

After label enconding, define `X` the independent variables and define `y` the dependent variable.

In [19]:
X = housedfcopy.drop("SalePrice", axis = 1)
y = housedfcopy["SalePrice"]

Split the data into a `train` and `test` dataset.

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [24]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2063, 85)
(516, 85)
(2063,)
(516,)


In [25]:
X_train.head()

,GrLivArea,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,ExteriorSF,AgeAtSale,TotalFullBath,TotalHalfBath,GarageAgeAtSale,Latitude,Longitude
1559,1516,20,5,100.0,10004,1,1,3,3,0,4,0,15,2,2,0,2,6,6,1964,1964,1,0,6,9,1,180.0,3,4,1,5,5,3,5,196.0,1,345.0,975.0,1516.0,1,4,1,3,1516,0,0,0.0,0.0,1,1,3,1,4,6,6,0,3,1,1,2.0,472.0,5,5,2,0,0,0,0,152,0,3,4,1,0,2,2009,9,4,8488,45,1.0,1.0,45.0,42.040791,-93.629457
413,1400,60,5,0.0,10274,1,1,0,3,0,1,0,23,2,2,0,5,6,7,1986,1986,1,0,12,13,1,141.0,3,2,1,2,5,3,5,331.0,6,0.0,345.0,676.0,1,4,1,3,698,702,0,0.0,0.0,2,1,3,1,4,6,6,0,3,1,1,2.0,465.0,5,5,2,0,48,0,0,0,0,3,4,1,0,7,2009,9,4,9576,23,2.0,1.0,23.0,42.035689,-93.685002
500,1324,50,6,50.0,6000,1,1,3,3,0,4,0,3,2,2,0,0,4,6,1936,1950,1,0,8,8,2,0.0,3,4,0,5,5,3,1,672.0,6,0.0,0.0,672.0,1,4,1,3,757,567,0,0.0,0.0,1,0,3,1,4,6,6,0,3,5,2,1.0,240.0,5,5,2,0,0,0,0,0,0,3,4,1,0,7,2009,9,4,5243,73,1.0,0.0,73.0,42.033504,-93.624532
361,1636,20,5,116.0,13501,1,1,0,3,0,0,0,24,2,2,0,2,8,5,2006,2006,1,0,12,13,3,208.0,2,4,2,2,5,3,2,63.0,6,0.0,1560.0,1623.0,1,0,1,3,1636,0,0,1.0,0.0,2,0,3,1,2,8,6,1,2,1,1,3.0,865.0,5,5,2,0,60,0,0,0,0,3,4,1,0,6,2007,9,4,11865,1,3.0,0.0,1.0,42.054482,-93.642598
1205,1232,30,5,64.0,8574,1,1,3,3,0,4,0,6,2,2,0,2,6,8,1916,2000,1,0,11,12,2,0.0,2,4,0,5,5,3,6,0.0,6,0.0,816.0,816.0,1,0,1,3,1232,0,0,0.0,0.0,1,0,3,1,2,6,6,1,2,5,2,2.0,440.0,5,5,2,0,0,180,0,0,0,3,4,1,0,10,2009,9,4,7342,93,1.0,0.0,93.0,42.017882,-93.646860


In [26]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()

In [27]:
rf.fit(X_train, y_train)

RandomForestRegressor()

In [ ]:
from sklearn.model_selection import GridSearchCV

## Define Grid 
grid = { 
    'n_estimators': [200,300,400,500],
    'max_features': ['sqrt','log2'],
    'max_depth' : [3,4,5,6,7],
    'random_state' : [18]
}

## Grid Search function
CV_rfr = GridSearchCV(estimator=RandomForestRegressor(), param_grid=grid, cv= 5)
CV_rfr.fit(X_train, y_train)

# Scratch Pad

In [77]:
pvals = {'Features': ['GrLivArea', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea',
       'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageFinish',
       'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive',
       'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'PoolQC', 'Fence', 'MiscFeature', 'MiscVal',
       'MoSold', 'YrSold', 'SaleType', 'SaleCondition', 'ExteriorSF',
       'AgeAtSale', 'TotalFullBath', 'TotalHalfBath', 'GarageAgeAtSale',
       'Latitude', 'Longitude'], 
         'P-Values': [0.00000000e+000, 1.70332217e-005, 1.56948479e-016, 1.26580346e-019,
       8.74659717e-046, 1.03945923e-002, 1.68387508e-008, 3.57361707e-053,
       5.15587677e-002, 8.37167349e-002, 1.35177201e-003, 6.18279092e-005,
       1.94152196e-022, 2.02422324e-006, 3.98096144e-002, 3.56290812e-003,
       7.60187304e-017, 0.00000000e+000, 2.65952084e-007, 4.12938360e-199,
       1.75299647e-174, 1.90116381e-031, 1.00915228e-007, 2.11464785e-009,
       3.63681149e-011, 1.94868666e-006, 4.17994428e-162, 3.67127284e-285,
       4.58872182e-009, 1.02190703e-097, 1.88286473e-207, 2.97490604e-008,
       9.53252314e-066, 1.91536641e-014, 6.29797485e-136, 1.09116658e-002,
       2.49295087e-001, 8.34265787e-018, 8.43913249e-313, 1.01919917e-004,
       4.09746259e-106, 2.10353013e-040, 5.22722871e-031, 2.04997469e-300,
       4.24274847e-042, 1.17548378e-001, 1.83850962e-051, 8.64519826e-002,
       4.30627469e-191, 1.08126717e-049, 8.72229088e-015, 6.49246739e-009,
       1.69268698e-243, 5.14533806e-156, 7.10110568e-008, 1.50136261e-154,
       8.32325670e-004, 2.04472901e-113, 4.36565786e-181, 1.44223822e-295,
       7.50724533e-291, 1.91697330e-028, 1.08180290e-036, 9.06457218e-044,
       5.79239062e-068, 2.84780525e-057, 3.43089553e-010, 1.12385746e-001,
       6.22323365e-010, 1.17288806e-001, 2.31611140e-006, 6.58092674e-013,
       1.09657964e-002, 3.38130720e-001, 5.18331026e-001, 9.48974724e-001,
       9.62617625e-001, 2.43654132e-013, 1.83737676e-037, 5.23762994e-199,
       3.37072156e-247, 1.19247252e-037, 1.17085615e-115, 8.74633218e-044,
       3.50537647e-036]}